# This notebook will create a static website based on the OWL files and data directory

## Create directories in documentation and quality_review directories

Use directory names in data for reference

In [90]:
import os
import shutil

def copy_folder_structure(source_dir, destination_dir):
    """
    Copy folder structure (only one level) from source directory to destination directory.
    Files are not copied.

    Args:
    - source_dir (str): The source directory to copy the folder structure from.
    - destination_dir (str): The destination directory to copy the folder structure to.
    """
    for root, dirs, files in os.walk(source_dir):
        for dir_name in dirs:
            source_path = os.path.join(root, dir_name)
            destination_path = os.path.join(destination_dir, os.path.relpath(source_path, source_dir))
            os.makedirs(destination_path, exist_ok=True)

# Example usage
source_directory = 'data'

copy_folder_structure(source_directory, 'quality_review')
copy_folder_structure(source_directory, 'documentation')


## Create documentation files 

embed iframe with LODE url 

- for each folder
  - create index.html
    - insert heading
    - insert iframe 
      - url = LODE service url plus url of ontology file 

In [91]:
import os

def generate_index_html(directory):
    """
    Generate an index.html file for each subfolder in the specified directory.

    Args:
    - directory (str): The directory containing subfolders.
    """
    for subdir in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, subdir)):
            index_html_path = os.path.join(directory, subdir, 'index.html')
            with open(index_html_path, 'w') as f:
                # Write HTML content to the index.html file
                f.write(f"""<!DOCTYPE html>
<html>
<head>
  <title>{subdir.capitalize()}</title>
  <style>
    body, html, iframe {{
      margin: 0;
      padding: 0;
      height: 100%;
      width: 100%;
      border: none;
    }}
  </style>
</head>
<body>
  <h1 style="text-transform: capitalize;">{subdir.capitalize()}</h1>
  <iframe src="http://150.146.207.114/lode/extract?lang=en&url=https%3A%2F%2Fwww.thegazette.co.uk%2Fdef%2F{subdir}" style="height: 100%; width: 100%;"></iframe>
</body>
</html>""")

# Specify the directory containing the subfolders
directory = 'documentation'

# Generate index.html files for each subfolder
generate_index_html(directory)


## Create quality review pages

- for each sub folder in quality_review
  - create summary page containing images if available from the data folder of 
    - class
        - relevance
        - completeness
      - object property
        - relevance
        - completeness
      - data property
        - relevance
        - completeness

  The data folder follows same structure as quality_review.
  The files are called completeness_class.jpg completeness_data_property.jpg and completeness_object_property.jpg 
  Similar file name pattern for relevance

In [92]:
import json

def generate_table_from_json(json_file):
    # Read JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)

        if "0" not in data:
            return ""
        

        html_output = "<table class='table table-striped datatable-table'>"

        # Write table headers
        html_output += "<tr>"
        for key in data["0"].keys():
            html_output += f"<th>{key}</th>"
        html_output += "</tr>\n"

        # Write table rows
        for item in data.values():
            html_output += "<tr>"
            for value in item.values():
                html_output += f"<td>{value}</td>"
            html_output += "</tr>\n"

        html_output += "</table>\n"

    return html_output




In [93]:
import json
%pip install prettytable
from prettytable import PrettyTable

def json_to_table(json_data):
    data = json.loads(json_data)

    table = PrettyTable()
    table.field_names = ["Type", "URI", "Label", "Comment", "Domain", "Range"]

    for category, items in data.items():
        for uri, properties in items.items():
            label = properties.get("label", "")
            comment = properties.get("comment", "")
            domain = properties.get("domain", "")
            range_ = properties.get("range", "")

            table.add_row([category.capitalize(), uri, label, comment, domain, range_])


    

    return table






DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [151]:
import os
import shutil

def create_summary_pages(root_dir):
    """
    Create summary pages for each subfolder in the quality_review directory.

    Args:
    - root_dir (str): The root directory containing the quality_review folder.
    """
    quality_review_dir = os.path.join(root_dir, 'quality_review')
    for folder_name in os.listdir(quality_review_dir):
        folder_path = os.path.join(quality_review_dir, folder_name)
        if os.path.isdir(folder_path):
            summary_path = os.path.join(folder_path, 'index.html')
            with open(summary_path, 'w') as summary_file:
                summary_file.write('<!DOCTYPE html>\n<html>\n<head> <link rel="stylesheet" href="../../lib/bootstrap-4.0.0-dist/css/bootstrap.min.css"><script src="../../lib/bootstrap-4.0.0-dist/js/bootstrap.min.js"></script><link rel="stylesheet" href="../../assets/css/style.css">\n<title>Summary</title>\n</head>\n<body>\n')

                summary_file.write(""" 
                <div class="container-fluid ">
    <div class="row justify-content-start ">
            <div class=" sidebar ">
                <p><img src="../../assets/images/logo.png" /> </p>
            </div>
            <div class="col-8 bg-info">
                <h1>The Gazette Data Model</h1>
            </div>
        </div>
        <div class="row justify-content-start">
            <div class=" bg-info sidebar">

                <ul class="list-group">

                    <li class="list-group-item"><a href="../../index.html">Dashboard</a></li>
                    <li class="list-group-item"><a href="../../quality_review/index.html">Quality review</a></li>
                    <li class="list-group-item"><a href="../../documentation/index.html">Documentation</a></li>

                </ul>
            </div>
            <div class="col">
""")


                summary_file.write(f'<h2 class="text-secondary">{folder_name.capitalize()} Ontology</h2>\n')

                summary_file.write("<h3 class='text-primary'>Documentation</h3><a class='text-info' target='_new' href=\"http://150.146.207.114/lode/extract?lang=en&url=https%3A%2F%2Fwww.thegazette.co.uk%2Fdef%2F" + folder_name +  "\">View documentation</a>")

                summary_file.write("<br/><br/><h3 class='text-primary'>Quality review</h3>")
                summary_file.write(f'<h4>OWL Class</h4>\n')

                summary_file.write(""" 
<div class="row">
                    <div class="col-sm-5">
                        <div class="card  ">
                            <div class="card-body ">
                                <h5 class="card-title">Relevance</h5>
                                """)
                image_relevance = f'data/{folder_name}/relevance_class.jpg'
                if os.path.exists(image_relevance):
                    summary_file.write(f'<img width="400" src="../../{image_relevance}" alt="Relevance">\n')

                summary_file.write(""" 
                                <p><a href="relevance.html"
                                    class="btn btn-info">Further information</a></p>
                            </div>
                        </div><br/>
                    </div>
                    <div class="col-sm-5 ">
                        <div class="card ">
                            <div class="card-body">
                                <h5 class="card-title">Completeness</h5>
                                """)
                image_completeness = f'data/{folder_name}/completeness_class.jpg'
                if os.path.exists(image_completeness):
                    summary_file.write(f'<img width="400" src="../../{image_completeness}" alt="Completeness Image">\n')
                    summary_file.write(""" <br/>
                                   <p><a href="completeness.html"
                                    class="btn btn-info">Further information</a></p>
                            </div></div><br/></div></div>
""" )

                
                
               
               
    
                summary_file.write(f'<h4>Object property</h4>\n')


                summary_file.write(""" 
<div class="row">
                    <div class="col-sm-5">
                        <div class="card  ">
                            <div class="card-body ">
                                <h5 class="card-title">Relevance</h5>
                                """)
                image_relevance = f'data/{folder_name}/relevance_object_property.jpg'
                if os.path.exists(image_relevance):
                    summary_file.write(f'<img width="400" src="../../{image_relevance}" alt="Relevance">\n')

                summary_file.write(""" 
                                <p><a href="relevance.html"
                                    class="btn btn-info">Further information</a></p>
                            </div>
                        </div><br/>
                    </div>
                    <div class="col-sm-5 ">
                        <div class="card ">
                            <div class="card-body">
                                <h5 class="card-title">Completeness</h5>
                                """)
                image_completeness = f'data/{folder_name}/completeness_object_property.jpg'
                if os.path.exists(image_completeness):
                    summary_file.write(f'<img width="400" src="../../{image_completeness}" alt="Completeness Image">\n')
                    summary_file.write(""" <br/>
                                   <p><a href="completeness.html"
                                    class="btn btn-info">Further information</a></p>
                            </div></div><br/></div></div>
""" )

                




                summary_file.write(f'<h2>Data property</h2>\n')


                summary_file.write(""" 
<div class="row">
                    <div class="col-sm-5">
                        <div class="card  ">
                            <div class="card-body ">
                                <h5 class="card-title">Relevance</h5>
                                """)
                image_relevance = f'data/{folder_name}/relevance_data_property.jpg'
                if os.path.exists(image_relevance):
                    summary_file.write(f'<img width="400" src="../../{image_relevance}" alt="Relevance">\n')

                summary_file.write(""" 
                                <p><a href="relevance.html"
                                    class="btn btn-info">Further information</a></p>
                            </div>
                        </div><br/>
                    </div>
                    <div class="col-sm-5 ">
                        <div class="card ">
                            <div class="card-body">
                                <h5 class="card-title">Completeness</h5>
                                """)
                image_completeness = f'data/{folder_name}/completeness_data_property.jpg'
                if os.path.exists(image_completeness):
                    summary_file.write(f'<img width="400" src="../../{image_completeness}" alt="Completeness Image">\n')
                    summary_file.write(""" <br/>
                                   <p><a href="completeness.html"
                                    class="btn btn-info">Further information</a></p>
                            </div></div><br/></div></div>
""" )





                

                

                summary_file.write(" </div></div>")
                summary_file.write('</body>\n</html>')
"""
                summary_data = f'data/{folder_name}/summary_data.json'
                
                if os.path.exists(summary_data):
                    table = json_to_table(summary_data)
                    
                    summary_file.write(table)
"""
                

# Specify the root directory
root_directory = ''

# Call the function to create summary pages
create_summary_pages(root_directory)


## Add links to home page 



In [105]:
import os

def generate_html_links_from_folders(root_dir):
    # Initialize an empty string to store HTML code
    html_code = "<ul>\n"

    html_code2 = ""

    # Traverse through the root directory
    for dir_name in os.listdir(root_dir):
        dir_path = os.path.join(root_dir, dir_name)
        if os.path.isdir(dir_path):
            # Append HTML code for each directory as a list item
            html_code += f"<li><a href=\"{dir_name}\">{dir_name}</a></li>\n"

            html_code2 += f"  <div class=\"col-sm-3\"><div class=\"card bg-warning\"><div class=\"card-body\"><h5 class=\"card-title\">{dir_name}</h5><p>View data model quality review and documentation</p><a href=\"quality_review/{dir_name}/index.html\" class=\"btn btn-sucess\">Quality review</a>&nbsp;<a href=\"documentation/{dir_name}/index.html\" class=\"btn \">Documentation</a></div></div><br/></div>"

    html_code += "</ul>"






    # Write HTML code to a file
    with open('links.html', 'w') as f:
        f.write(html_code)
        f.write(html_code2)
# Example usage
root_directory = 'data'
generate_html_links_from_folders(root_directory)
